<a href="https://colab.research.google.com/github/toshikifu/shift_recorder/blob/main/%E8%B3%AA%E5%95%8F%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ドライブへ接続

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/toshiki/T5'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/toshiki/T5


# 依存ライブラリのインストール

In [ ]:
!pip install -qU torch==1.13.* torchtext==0.14.* torchvision==0.14.* torchaudio==0.13.* torchmetrics==0.11.* \
    transformers==4.26.1 pytorch_lightning==1.9.3 sentencepiece==0.1.97

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 126.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip list | grep -e "torch" -e "transformers" -e "sentencepiece"

pytorch-lightning             1.9.3
sentencepiece                 0.1.97
torch                         1.13.1
torchaudio                    0.13.1
torchdata                     0.6.1
torchmetrics                  0.11.4
torchsummary                  1.5.1
torchtext                     0.14.1
torchvision                   0.14.1
transformers                  4.26.1


# 各種ディレクトリの作成
- data : 学習用データセット格納
- model : 学習地味モデル格納

In [ ]:
!mkdir -p ./content/data ./content/model

In [ ]:
# 学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 転移学習済みモデルディレクトリ
MODEL_DIR = "./content/model"

# 学習用データ用意

In [ ]:
# 運転ドメインのQAデータを使う
import json

json_file = open("./DDQA-1.0_RC-QA_train.json")
json_object = json.load(json_file)

print(json_object["data"][0]["paragraphs"][0].items())
print(len(json_object["data"][0]["paragraphs"]))

dict_items([('context', '自転車はドコ走りゃイイのさ！自転車はそもそも車道の左端を走るのがルールなのですが、いざ車道を走るとものすごい車に邪険にされます。クロスバイクで往復３２ｋｍ程の道のりを自転車通勤してた時の事。赤信号で左側に自転車が止められないように左に幅寄せしてきたりとか、渋滞してる時に抜きつ抜かれつになるのが嫌なのか、やっぱり幅寄せされる事がよくあります。'), ('qas', [{'id': '57590011580005_00', 'question': '自転車が走るのは車道のどちら側がルールですか？', 'answers': [{'text': '左端', 'answer_start': 25}, {'text': '左端を走る', 'answer_start': 25}, {'text': '車道の左端を走るのがルール', 'answer_start': 22}], 'is_impossible': False}, {'id': '57590011580005_01', 'question': '自転車はどこを走るのがルールですか？', 'answers': [{'text': '車道の左端', 'answer_start': 22}, {'text': '車道の左端を走る', 'answer_start': 22}, {'text': '車道の左端を走るのがルール', 'answer_start': 22}], 'is_impossible': False}])])
8968


# データ形式の変換
| id | body | question | answer |
| -- | ---- | -------- | ------ |
| 01 | text | 質問     | 回答   |

In [ ]:
body = json_object["data"][0]["paragraphs"][0]["context"]
question = json_object["data"][0]["paragraphs"][0]["qas"][0]["question"]
answer = json_object["data"][0]["paragraphs"][0]["qas"][0]["answers"][-1]["text"]

print(f"""
本文 : {body}
質問 : {question}
回答 : {answer}
""")


本文 : 自転車はドコ走りゃイイのさ！自転車はそもそも車道の左端を走るのがルールなのですが、いざ車道を走るとものすごい車に邪険にされます。クロスバイクで往復３２ｋｍ程の道のりを自転車通勤してた時の事。赤信号で左側に自転車が止められないように左に幅寄せしてきたりとか、渋滞してる時に抜きつ抜かれつになるのが嫌なのか、やっぱり幅寄せされる事がよくあります。
質問 : 自転車が走るのは車道のどちら側がルールですか？
回答 : 車道の左端を走るのがルール



In [ ]:
all_data = []

for i in range(len(json_object["data"][0]["paragraphs"])):
  body = json_object["data"][0]["paragraphs"][i]["context"]
  question = json_object["data"][0]["paragraphs"][i]["qas"][0]["question"]
  answer = json_object["data"][0]["paragraphs"][i]["qas"][0]["answers"][-1]["text"]

  all_data.append({
      "id": i+1,
      "body": body,
      "question": question,
      "answer": answer
  })

In [ ]:
len(all_data)

8968

# データ分割
＊データセットに学習用、テスト用、検証用があるが短時間でやりたいのでテストをさらに分割して実施する。

90% : 5% : 5% の比率で train / dev / test に分ける

In [ ]:
import random
from tqdm import tqdm

random.seed(42)
random.shuffle(all_data)

def to_line(data):
    question = data["question"]
    body = data["body"]
    id = data["id"]

    assert len(question) > 0 and len(body) > 0
    return f"{question}\t{body}\t{id}\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.9, 0.05, 0.05

with open(f"./content/data/train.tsv", "w", encoding="utf-8") as f_train, \
    open(f"./content/data/dev.tsv", "w", encoding="utf-8") as f_dev, \
    open(f"./content/data/test.tsv", "w", encoding="utf-8") as f_test:
    
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

8968it [00:00, 249754.45it/s]


In [ ]:
# データの確認
!head -3 ./content/data/test.tsv

何回事故にあった？	へーっ、バンパーって凹むんじゃなくて穴が開くんだー。と、感心はせずにヘコタレました。世の中には車の運転に向いてない人が存在します。この事故の前に、３回ほど事故にあっているらしい。	6292
滝バカは誰か？	滝は引っ込んでいるから、ただ観たんじゃ見えないけど、こうやって見れば見えるよ」といって、足を一歩前に出して、覗き込むしぐさをして見せてくれた。何とおばあさんの言うには、この集落のはずれの橋から向こうは、その滝の近くの橋しか途中には橋はないという。なんと言うことだ、私が諦めて帰ってきた橋のすぐ向こうに不老の滝はあったということになる。さすがの「滝バカ」の私も、もう一度あの川原を歩く気にはならなかった。	3186
奥様と話をしたり、あと何をしましたか？	外を闇雲に歩いて疲れた足をいたわりつつ、奥様と話をしたり、お茶呑んだり、ボーッとしたり。気にしないと早く過ぎるのに、待っていると中々進まないのが時間だ。それでもちゃんと時間は進み、ほぼ定刻に来た阿寒バスに乗って羅臼に無事到着したのだった。バス停は前がお寺、向かいが今日泊まる高島屋旅館という絶好のロケーションだった。	6791


# 学習に必要なクラスなどを定義
学習にはPyTorch / PyTorch-lightning / transformers を利用する

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir = "./content/data",  # データセットのディレクトリ
    model_name_or_path = PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path = PRETRAINED_MODEL_NAME,

    learning_rate = 3e-4,
    weight_decay = 0.0,
    adam_epsilon = 1e-8,
    warmup_steps = 0,
    gradient_accumulation_steps = 1,

    # max_input_length = 512,
    # max_target_length = 64,
    # train_batch_size = 8,
    # eval_batch_size = 8,
    # num_train_epochs = 4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    max_grad_norm = 1.0,
    seed = 42,
)

In [ ]:
class TsvDataset(Dataset):
  def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
    self.file_path = os.path.join(data_dir, type_path)

    self.input_max_len = input_max_len
    self.target_max_len = target_max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []

    self._build()

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    source_mask = self.inputs[index]["attention_mask"].squeeze()
    target_mask = self.targets[index]["attention_mask"].squeeze()

    return {"source_ids": source_ids, "source_mask": source_mask,
            "target_ids": target_ids, "target_mask": target_mask}

  def _make_record(self, question, body, id):
    input = f"{body}"
    target = f"{question}"
    return input, target

  def _build(self):
    with open(self.file_path, "r", encoding="utf-8") as f:
      for line in f:
        line = line.strip().split("\t")
        # assert len(line) == 3
        assert len(line[0]) > 0
        assert len(line[1]) > 0
        assert len(line[2]) > 0

        question = line[0]
        body = line[1]
        id = line[2]

        input, target = self._make_record(question, body, id)

        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input],
            max_length = self.input_max_len,
            truncation = True,
            padding = "max_length",
            return_tensors = "pt"
        )

        tokenized_targets = self.tokenizer.batch_encode_plus(
            [target],
            max_length = self.target_max_len,
            truncation = True,
            padding = "max_length",
            return_tensors = "pt"
        )

        self.inputs.append(tokenized_inputs)
        self.targets.append(tokenized_targets)


In [ ]:
# tokenizer(sentencepiece)モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)

#テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "train.tsv", input_max_len=512, target_max_len=512)

In [ ]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
とは考えない神経が信じられないです。他にも大型スーパー等で同じように駐車する自動車も。偽善者ぶるつもりはありませんが、何のための障害者専用スペースなのかを理解できる常識を身につけて欲しいものです。バイクを目の敵にする自動車がいますので、十分に注意して下さい。</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.hparams.eval_batch_size, 
                          num_workers=4)


# 転移学習の実装

In [ ]:
# ハイパラの設定
args_dict.update({
    "max_input_length": 512,
    "max_target_length": 64,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "num_train_epochs": 8
})
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches = args.gradient_accumulation_steps,
    gpus = args.n_gpu,
    max_epochs = args.num_train_epochs,
    precision = 16 if args.fp_16 else 32,
    amp_level = "01",
    amp_backend = "apex",
    gradient_clip_val = args.max_grad_norm,
)

In [ ]:
# 転移学習の実行
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保持
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:402: LightningDeprecationWarning: The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside PyTorch Lightning has been deprecated in v1.9.0 and will be removed in v2.0.0. The `Trainer(amp_backend='apex')` argument is deprecated. Removing this argument will avoid this message, it will select PyTorch's implementation automatically.
  rank_zero_deprecation(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:413: LightningDeprecationWarning: The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside PyTorch Lightning has been deprecated in v1.9.0 and will be removed in v2.0.0. The `Trainer(amp_level='01')` argument is deprecated. Removing this argument will avoid this message.
  rank_zero_deprecation(
/usr/local/lib/python3.10/dist-packages/pytorch_light

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=8` reached.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

In [ ]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

trained_model.eval()

inputs = []
outputs = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
        repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False) 
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)


  0%|          | 0/56 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
for output, target, input in zip(outputs, targets, inputs):
    print("generated: " + output)
    print("actual:    " + target)
    print("body:      " + input)
    print()

# 医療データで質問生成

In [ ]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "medical_test_data.tsv", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

trained_model.eval()

inputs = []
outputs = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
        repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False) 
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)


  0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
for output, target, input in zip(outputs, targets, inputs):
    print("generated: " + output)
    print("actual:    " + target)
    print("body:      " + input)
    print()

generated: 何が体の内外の情報を感受し、これに反応して行動するという重要な役割を果たしているのか?
actual:    テストです
body:      神経系は、体の内外の情報を感受し、これに反応して行動するという重要な役割を果たしています。

generated: 内蔵の働きを調節する何機関であるか?
actual:    テストです
body:      また、内蔵の働きを調節する統制機関でもあります。

generated: 神経系は、何と何に分けられていますか?
actual:    テストです
body:      神経系は、中枢神経(脳、脊髄)と末梢神経に分けられます。

generated: 中枢神経は、何から送られてくる情報を受け取り、これに対する反応を命令として末梢に伝えますか?
actual:    テストです
body:      中枢神経は、末梢神経から送られてくる情報を受け取り、これに対する反応を命令として末梢に伝えます。

generated: 脳は、大脳、小脳、脳幹(間脳・中脳・橋・延髄)に分けられていますか?
actual:    テストです
body:      脳は、大脳、小脳、脳幹(間脳・中脳・橋・延髄)に分けられます。

generated: 言語中枢があるのは?
actual:    テストです
body:      大脳は左右二つの半球があり、言語中枢がある左半球は言語を理解したり、言語を出す指令を出したりします。

generated: 何が空間的時間的関係を認識するという、重要な枠割を果たしていますか?
actual:    テストです
body:      右半球は空間的時間的関係を認識するという、重要な枠割を果たしています。

generated: 小脳は、姿勢を保つ、歩行など運動の調節に関与しているのは何か?
actual:    テストです
body:      小脳は、姿勢を保つ、歩行など運動の調節に関与しています。

generated: 生命維持に重要な役割をはたしているのは?
actual:    テストです
body:      脳幹には、心臓や血管の運動、呼吸などの中枢があり、意識と関係する網様体賦活系も存在するなど、生命維持に大切な役割をもっています。

generated: 運動系